In [ ]:
import tensorflow as tf
from tensorflow import keras
import os
from LodingData import LodingData
from DataGenerator import DataGenerator
from keras.models import Sequential, Model, Input
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout, concatenate, Lambda, BatchNormalization
from keras import metrics
from tensorflow.keras.optimizers import Adam

In [ ]:
%tensorflow_version 2.x
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
from google.colab import drive
drive.flush_and_unmount()

Drive not mounted, so nothing to flush and unmount.


In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data_path="/content/drive/My Drive/Colab Notebooks/face_verification/data"
LodingData(data_path)

extracting_training_images.....
extracting_finished!
extracting_training_images.....
extracting_finished!
serialization of training data finished
serialization of validation data finished


In [ ]:
wights_path=r"/content/drive/My Drive/Colab Notebooks/face_verification/code"
model=keras.models.load_model(os.path.join(wights_path, 'resnet50_triplet_loss_2048.h5'))
# model.summary()

In [ ]:
in_anc = Input(shape=(224,224,3))
in_pos = Input(shape=(224,224,3))
in_neg = Input(shape=(224,224,3))

em_anc = model(in_anc)
em_pos = model(in_pos)
em_neg = model(in_neg)

out = concatenate([em_anc, em_pos, em_neg], axis=1)

siamese_net = Model(
    [in_anc, in_pos, in_neg],
    out
)

In [ ]:
def triplet_loss(alpha, emb_dim):
    def loss(y_true, y_pred):
        anc, pos, neg = y_pred[:,:emb_dim], y_pred[:,emb_dim:2*emb_dim], y_pred[:,2*emb_dim:]
        distance1 = tf.keras.losses.cosine_similarity(anc, pos)
        distance2 = tf.keras.losses.cosine_similarity(anc, neg)
        return tf.keras.backend.clip(distance1 - distance2 + alpha, 0., None)
    return loss

In [ ]:
siamese_net.compile(optimizer=Adam(lr=.0001),loss=triplet_loss(alpha=0.2, emb_dim=2048))



history=siamese_net.fit(x=DataGenerator(data_path,1000,2048,True)[0][0],
                        y=DataGenerator(data_path,1000,2048,True)[0][1],
                    validation_data=(DataGenerator(data_path,100,2048,False)[0][0],
                                     DataGenerator(data_path,100,2048,False)[0][1])
                    ,epochs=50,
                    use_multiprocessing=True,verbose=2)

Epoch 1/50
32/32 - 62s - loss: 0.0089 - val_loss: 0.0170
Epoch 2/50
32/32 - 18s - loss: 8.5480e-04 - val_loss: 0.0164
Epoch 3/50
32/32 - 18s - loss: 1.4877e-04 - val_loss: 0.0164
Epoch 4/50
32/32 - 18s - loss: 6.0105e-06 - val_loss: 0.0163
Epoch 5/50
32/32 - 18s - loss: 0.0000e+00 - val_loss: 0.0162
Epoch 6/50
32/32 - 18s - loss: 0.0000e+00 - val_loss: 0.0162
Epoch 7/50
32/32 - 18s - loss: 0.0000e+00 - val_loss: 0.0162
Epoch 8/50
32/32 - 18s - loss: 0.0000e+00 - val_loss: 0.0162
Epoch 9/50
32/32 - 18s - loss: 0.0000e+00 - val_loss: 0.0162
Epoch 10/50
32/32 - 18s - loss: 0.0000e+00 - val_loss: 0.0162
Epoch 11/50
32/32 - 18s - loss: 0.0000e+00 - val_loss: 0.0162
Epoch 12/50
32/32 - 18s - loss: 0.0000e+00 - val_loss: 0.0162
Epoch 13/50
32/32 - 18s - loss: 0.0000e+00 - val_loss: 0.0162
Epoch 14/50
32/32 - 18s - loss: 0.0000e+00 - val_loss: 0.0162
Epoch 15/50
32/32 - 18s - loss: 0.0000e+00 - val_loss: 0.0162
Epoch 16/50
32/32 - 18s - loss: 0.0000e+00 - val_loss: 0.0162
Epoch 17/50
32/32 - 1

In [ ]:
if os.path.isfile(os.path.join(wights_path, 'new_resnet50_triplet_loss_2048.h5')) is False:
  model.save(os.path.join(wights_path, 'new_resnet50_triplet_loss_2048.h5'))

In [ ]:
wights_path=r"/content/drive/My Drive/Colab Notebooks/face_verification/code"
model=keras.models.load_model(os.path.join(wights_path, 'new_resnet50_triplet_loss_2048.h5'))

In [ ]:
def is_match(known_embedding, candidate_embedding, thresh=0.5):
	# calculate distance between embeddings
	score = cosine(known_embedding, candidate_embedding)
	if score <= thresh:
		return True
	else:
		return False

In [ ]:
model.compile()
data_path="/content/drive/My Drive/Colab Notebooks/face_verification/data"
X=DataGenerator(data_path2,1000,2048,True)[0][0]
j=0
for i in range(1000):
  img1 = tf.convert_to_tensor(cv2.resize(X[0][i],(224,224)))
  img2 = tf.convert_to_tensor(cv2.resize(X[1][i],(224,224)))
  img3 = tf.convert_to_tensor(cv2.resize(X[2][i],(224,224)))
  y1=model.predict(np.expand_dims(img1, axis=0))
  y2=model.predict(np.expand_dims(img2, axis=0))
  y3=model.predict(np.expand_dims(img3, axis=0))
  if is_match(y1,y2):
    j+=1
  if not is_match(y2,y3):
    j+=1
  if not is_match(y1,y3):
    j+=1
print(j)
accuracy=(j/3000)*100
print(accuracy)


NameError: ignored